# Black-Scholes Implementation

## Option Pricing

Reference: https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#Black%E2%80%93Scholes_formula

In [1630]:
import numpy as np

from scipy.stats import norm

interest_rate = 0.00
underlying_price = 100
strike_price = 105
time = 30.0 / 365.0
implied_volatility = 0.30

def black_scholes(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type: str):
    d1 = (np.log(underlying_price / strike_price) + (interest_rate + implied_volatility**2/2)*time)/(implied_volatility * np.sqrt(time))
    d2 = d1 - (implied_volatility * np.sqrt(time))
    if contract_type.lower() == 'c':
        option_price = (underlying_price * norm.cdf(d1, 0, 1)) - (strike_price * np.exp(-interest_rate * time) * norm.cdf(d2, 0, 1))
    else:
        option_price = strike_price * np.exp(-interest_rate * time) * norm.cdf(-d2, 0, 1) - underlying_price*norm.cdf(-d1, 0, 1)
    return option_price

In [1631]:
print('Option Price:', round(black_scholes(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))

Option Price: 1.57


### Validation

In [1632]:
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho

In [1633]:
print('Validation Price:', round(bs('c', underlying_price, strike_price, time, interest_rate, implied_volatility), 2))

Validation Price: 1.57


## Option Greeks

Reference: https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#The_Options_Greeks

### Delta

Delta measures the rate of change of the theorectical option value with respect to a change in the underlying.

$\Delta = \frac{\partial V}{\partial S}$

$\Delta_{call} = \Phi (d1)$

$\Delta_{put} = -\Phi (-d1)$


In [1634]:
def black_scholes_delta(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type):
    d1 = (np.log(underlying_price / strike_price) + (interest_rate + implied_volatility**2/2)*time)/(implied_volatility * np.sqrt(time))
    if contract_type == 'C':
        delta = norm.cdf(d1, 0, 1)
    else:
        delta = -norm.cdf(-d1, 0, 1)
    return delta

In [1635]:
print('Option Delta:', round(black_scholes_delta(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))

Option Delta: 0.3


#### Validation

In [1636]:
print('Validation Delta:', round(delta('c', underlying_price, strike_price, time, interest_rate, implied_volatility), 2))

Validation Delta: 0.3


### Gamma

Gamma measures the rate of change of delta with respect to changes in the underlying.

$\Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}$

$\Gamma = \frac{\phi (d1)}{S \sigma \sqrt{r}}$

In [1637]:
def black_scholes_gamma(interest_rate, underlying_price, strike_price, time, implied_volatility):
    d1 = (np.log(underlying_price / strike_price) + (interest_rate + implied_volatility**2/2)*time)/(implied_volatility * np.sqrt(time))
    return norm.pdf(d1, 0, 1) / (underlying_price * implied_volatility * np.sqrt(time))

In [1638]:
print('Option Gamma:', round(black_scholes_gamma(interest_rate, underlying_price, strike_price, time, implied_volatility), 2))

Option Gamma: 0.04


#### Validation

In [1639]:
print('Validation Gamma:', round(gamma('c', underlying_price, strike_price, time, interest_rate, implied_volatility), 2))

Validation Gamma: 0.04


### Vega

Vega measures the sensitivity of an option to volatility, the change in option value with respect to the volatility of the underlying.

$\nu = \frac{\partial V}{\partial \sigma}$

$\nu = S \phi (d1) \sqrt{\tau}$

In [1640]:
def black_scholes_vega(interest_rate, underlying_price, strike_price, time, implied_volatility):
    d1 = (np.log(underlying_price / strike_price) + (interest_rate + implied_volatility**2/2)*time)/(implied_volatility * np.sqrt(time))
    return underlying_price * norm.pdf(d1, 0, 1) * np.sqrt(time) * 0.01 # Sensitivity to a 1% change.

In [1641]:
print('Option Vega:', round(black_scholes_vega(interest_rate, underlying_price, strike_price, time, implied_volatility), 2))

Option Vega: 0.1


#### Validation

In [1642]:
print('Validation Vega:', round(vega('c', underlying_price, strike_price, time, interest_rate, implied_volatility), 2))

Validation Vega: 0.1


### Theta

Theta measures the sensitivity of the value of the option to the passage of time.

$\Theta = -\frac{\partial V}{\partial \tau}$

$\Theta_{call} = -\frac{S \phi (d1) \sigma}{2 \sqrt{\tau}} - r K exp(-rT) \Phi (d2)$

$\Theta_{put} = -\frac{S \phi (d1) \sigma}{2 \sqrt{\tau}} + r K exp(-rT) \Phi (-d2)$

In [1643]:
def black_scholes_theta(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type):
    d1 = (np.log(underlying_price / strike_price) + (interest_rate + implied_volatility**2/2)*time)/(implied_volatility * np.sqrt(time))
    d2 = d1 - (implied_volatility * np.sqrt(time))
    if contract_type == 'C':
        theta = -((underlying_price * norm.pdf(d1, 0, 1) * implied_volatility) / (2 * np.sqrt(time))) - (interest_rate * strike_price * np.exp(-interest_rate * time) * norm.cdf(d2, 0, 1))
    else:
        theta = -((underlying_price * norm.pdf(d1, 0, 1) * implied_volatility) / (2 * np.sqrt(time))) + (interest_rate * strike_price * np.exp(-interest_rate * time) * norm.cdf(-d2, 0, 1))
    return theta / 365 # Daily change.

In [1644]:
print('Option Theta:', round(black_scholes_theta(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))

Option Theta: -0.05


#### Validation

In [1645]:
print('Validation Theta:', round(theta('c', underlying_price, strike_price, time, interest_rate, implied_volatility), 2))

Validation Theta: -0.05


### Rho

Rho measures the sensitivity of the value of the option to the interest rate.

$\rho = \frac{\partial V}{\partial r}$

$\rho_{call} = K \tau exp(-rT) \Phi (d2)$

$\rho_{put} = -K \tau exp(-rT) \Phi (-d2)$

In [1646]:
def black_scholes_rho(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type):
    d1 = (np.log(underlying_price / strike_price) + (interest_rate + implied_volatility**2/2)*time)/(implied_volatility * np.sqrt(time))
    d2 = d1 - (implied_volatility * np.sqrt(time))
    if contract_type == 'C':
        rho = underlying_price * time * np.exp(-interest_rate * time) * norm.cdf(d2, 0, 1)
    else:
        rho = -underlying_price * time * np.exp(-interest_rate * time) * norm.cdf(-d2, 0, 1)
    return rho * 0.01 # Sensitivity to a 1% change.

In [1647]:
print('Option Rho:', round(black_scholes_rho(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))

Option Rho: 0.02


#### Validation

In [1648]:
print('Validation Rho:', round(rho('c', underlying_price, strike_price, time, interest_rate, implied_volatility), 2))

Validation Rho: 0.02


### Summary

In [1649]:
print('Option Price:', round(black_scholes(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))
print('       Delta:', round(black_scholes_delta(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))
print('       Gamma:', round(black_scholes_gamma(interest_rate, underlying_price, strike_price, time, implied_volatility), 2))
print('        Vega:', round(black_scholes_vega(interest_rate, underlying_price, strike_price, time, implied_volatility), 2))
print('       Theta:', round(black_scholes_theta(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))
print('         Rho:', round(black_scholes_rho(interest_rate, underlying_price, strike_price, time, implied_volatility, contract_type='C'), 2))

Option Price: 1.57
       Delta: 0.3
       Gamma: 0.04
        Vega: 0.1
       Theta: -0.05
         Rho: 0.02


## Implied Volatility Calculation

In [1650]:
def black_scholes_implied_volatility(option_price, underlying_price, strike_price, time_to_maturity, interest_rate, contract_type, tolerance=0.0001):
    _implied_volatility = 0.20 # Initial guess of 20%.
    while True: # Assuming convergence of Newton's method.
        bs_price = black_scholes(interest_rate, underlying_price, strike_price, time_to_maturity, _implied_volatility, contract_type)
        bs_vega = black_scholes_vega(interest_rate, underlying_price, strike_price, time, _implied_volatility) * 100
        price_discrepancy = bs_price - option_price
        updated_implied_volatility = _implied_volatility - (price_discrepancy / bs_vega)
        if ((abs(updated_implied_volatility - _implied_volatility) < tolerance) or (abs(price_discrepancy) < tolerance)):
            return updated_implied_volatility
        _implied_volatility = updated_implied_volatility

In [1651]:
print('Implied Volatility: {:0.2f}'.format(round(black_scholes_implied_volatility(1.70, 4504, 4375, (1 / 365), interest_rate, contract_type='p'), 2)))

Implied Volatility: 0.34
